In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[4]") \
    .appName("Walamart Ecommerce") \
    .getOrCreate()

22/10/20 12:37:46 WARN Utils: Your hostname, 1011000011101110 resolves to a loopback address: 127.0.1.1; using 192.168.1.72 instead (on interface wlp3s0)
22/10/20 12:37:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/20 12:37:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/20 12:37:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [52]:
df = spark.read.json("walmart_ecommerce_product_details.json")

In [4]:
df.printSchema()

root
 |-- Available: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Crawl Timestamp: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Gtin: string (nullable = true)
 |-- Item Number: string (nullable = true)
 |-- List Price: string (nullable = true)
 |-- Package Size: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Product Url: string (nullable = true)
 |-- Sale Price: string (nullable = true)
 |-- Uniq Id: string (nullable = true)
 |-- _corrupt_record: string (nullable = true)



In [6]:
df.show(10)

+---------+--------------------+--------------------+--------------------+--------------------+------------+-----------+----------+------------+-----------+--------------------+--------------------+----------+--------------------+---------------+
|Available|               Brand|            Category|     Crawl Timestamp|         Description|        Gtin|Item Number|List Price|Package Size|Postal Code|        Product Name|         Product Url|Sale Price|             Uniq Id|_corrupt_record|
+---------+--------------------+--------------------+--------------------+--------------------+------------+-----------+----------+------------+-----------+--------------------+--------------------+----------+--------------------+---------------+
|     TRUE|       In Style Eyes|Health|Home Healt...|2019-03-22 17:10:...|Stunning Looking ...|096647824338|           |     19.99|            |           |In Style Eyes Cat...|https://www.walma...|     19.99|459b05f3cb7f1cba0...|           null|
|     TRUE| 

In [7]:
df.toPandas().to_csv('OutputDatas/walmart_ecommerce_product_details.csv',index= False)

# 1. Get the Brand along with products associated with it.

In [20]:
brand_with_product = df.select('Brand','Product Name')

In [21]:
brand_with_product.show(truncate = False)

+--------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Brand               |Product Name                                                                                                                                                              |
+--------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|In Style Eyes       |In Style Eyes Cateye Two Tone Reading Glasses                                                                                                                             |
|In Style Eyes       |In Style Eyes Cateye Two Tone Reading Glasses                                                                                                                             |
|In Style Eyes       |In Style

In [22]:
brand_with_product = brand_with_product.distinct()

In [23]:
brand_with_product.show(truncate = False)

+-----------------------+-----------------------------------------------------------------------------------------------------+
|Brand                  |Product Name                                                                                         |
+-----------------------+-----------------------------------------------------------------------------------------------------+
|MansBestFriend         |2 Level Lounge Activity Center with Retreat Hide Away Corduroy                                       |
|SmartGifts             |Gemstone Globe Bright Silver Keychain - Marine Blue Ocean                                            |
|CHOSEN SUPPLIES        |Replacement for LINCOLN MARK LT YEAR 2008 LICENSE PLATE LIGHT 10 PACK replacement light bulb lamp    |
|CHOSEN SUPPLIES        |Replacement for AGILENT / HP AA475 HOLLOW CATHODE LAMP BORON B                                       |
|CHOSEN SUPPLIES        |Replacement for STERLING 827 YEAR 1989 BRAKE LIGHT YELLOW LED REPLACEMENT repla

In [24]:
brand_with_product.toPandas().to_csv('OutputDatas/1. brand_with_product.csv',index= False)

# 2. List all the product names whose list price is greater than sales price

In [25]:
list_price_greater_than_sales_price = df.select('Product Name','List Price','Sale Price')

In [26]:
list_price_greater_than_sales_price.show()

+--------------------+----------+----------+
|        Product Name|List Price|Sale Price|
+--------------------+----------+----------+
|In Style Eyes Cat...|     19.99|     19.99|
|In Style Eyes Cat...|     19.99|     19.99|
|In Style Eyes Cat...|     19.99|     19.99|
|In Style Eyes Cat...|     19.99|     19.99|
|Upper Crust Carib...|     39.96|     39.96|
|Fish Tank Ornamen...|    199.99|       100|
|3-Pack Black Peel...|     19.54|     19.54|
|Replacement for L...|     32.51|     32.51|
|American Foxhound...|     10.59|     10.59|
|Ironwood Modular ...|       680|    419.99|
|Classique 766 Pos...|        56|        56|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walk

In [27]:
list_price_greater_than_sales_price.printSchema()

root
 |-- Product Name: string (nullable = true)
 |-- List Price: string (nullable = true)
 |-- Sale Price: string (nullable = true)



In [30]:
from pyspark.sql.functions import col

In [31]:
list_price_greater_than_sales_price = list_price_greater_than_sales_price.withColumn("List Price",col("List Price").cast("float"))
list_price_greater_than_sales_price = list_price_greater_than_sales_price.withColumn("Sale Price",col("Sale Price").cast("float"))

In [32]:
list_price_greater_than_sales_price.printSchema()

root
 |-- Product Name: string (nullable = true)
 |-- List Price: float (nullable = true)
 |-- Sale Price: float (nullable = true)



In [34]:
list_price_greater_than_sales_price = list_price_greater_than_sales_price.filter(df['List Price']>df['Sale Price'])

In [35]:
list_price_greater_than_sales_price.show()

+--------------------+----------+----------+
|        Product Name|List Price|Sale Price|
+--------------------+----------+----------+
|Fish Tank Ornamen...|    199.99|     100.0|
|Ironwood Modular ...|     680.0|    419.99|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walker AW St...|     46.99|     14.29|
|Ames Walk

In [38]:
list_price_greater_than_sales_price.count()

4279

In [37]:
list_price_greater_than_sales_price.distinct().count()

912

In [39]:
disttinct_list_price_greater_than_sales_price = list_price_greater_than_sales_price.distinct()

In [40]:
disttinct_list_price_greater_than_sales_price.show()

+--------------------+----------+----------+
|        Product Name|List Price|Sale Price|
+--------------------+----------+----------+
|Super Mario Bros ...|     31.38|     20.99|
|Buckle Down DC Co...|     19.99|     14.99|
|Doco DCS005-10XL ...|     23.13|     17.69|
|CHOCOLATE CORDIAL...|     21.99|     19.71|
|Yosoo 32 Types 20...|      7.49|      4.99|
|Fish Tank Ornamen...|    199.99|     100.0|
|Enchanted Shimmer...|      79.0|     53.99|
|Letter U Football...|      14.0|     11.59|
|Rayne Mirrors Ame...|    636.35|    471.27|
|Sigvaris Secure 5...|    131.95|    105.56|
|Gemstone Globe Br...|     17.77|     13.06|
|Color Packing Tap...|     31.95|     27.16|
|Buckle Down Buckl...|     34.99|     23.95|
|Ishow 7A Brazilia...|    224.18|    118.82|
|Rayne Mirrors Ame...|     794.6|    588.49|
|LAMINATED POSTER ...|     16.99|     15.29|
|Stain Cleaner&nbs...|     42.89|     28.59|
|Eros ATT882930 Kn...|    478.63|    354.44|
|Areo Home Wall Bu...|     81.99|     36.96|
|Women's B

In [41]:
disttinct_list_price_greater_than_sales_price.toPandas().to_csv('OutputDatas/2. list_price_greater_than_sales_price.csv',index= False)

# 3. Count the number of product names whose list price is greater than sales price

In [44]:
list_price_greater_than_sales_price.select('Product Name').distinct().count()

544

# 4. List all the products belong to a “women” category.

In [91]:
products_belongs_women_category = df.where(col('Category').contains('Women') | col('Category').contains('women'))
products_belongs_women_category = products_belongs_women_category.select('Product Name','Category')
products_belongs_women_category.show(5)

+--------------------+--------------------+
|        Product Name|            Category|
+--------------------+--------------------+
|Classique 766 Pos...|Clothing|Women|Wo...|
|Ames Walker AW St...|Clothing|Women|Wo...|
|Ames Walker AW St...|Clothing|Women|Wo...|
|Ames Walker AW St...|Clothing|Women|Wo...|
|Ames Walker AW St...|Clothing|Women|Wo...|
+--------------------+--------------------+
only showing top 5 rows



In [92]:
products_belongs_women_category.count()

3221

In [93]:
products_belongs_women_category.distinct().count()

114

In [96]:
distinct_products_belongs_women_category = products_belongs_women_category.distinct()

distinct_products_belongs_women_category.show()

+--------------------+--------------------+
|        Product Name|            Category|
+--------------------+--------------------+
|Classique 766 Pos...|Clothing|Women|Wo...|
|Celtic Knot Weave...|Clothing|Bags & A...|
|Owl shaped keycha...|Clothing|Bags & A...|
|Secret Key Hider,...|Clothing|Bags & A...|
|Monarch M-initial...|Clothing|Bags & A...|
|Ramones Plastic K...|Clothing|Bags & A...|
|  Stanford Key Chain|Clothing|Bags & A...|
|The Wonder Years ...|Clothing|Women|Wo...|
|Handmade Glass Pe...|Jewelry|Womens Je...|
|Instant Gift Phot...|Clothing|Bags & A...|
|Retro Owl Time Ge...|Clothing|Bags & A...|
|Keychain - F-16 F...|Clothing|Bags & A...|
|Diabetic Bamboo Q...|Clothing|Women|Wo...|
|KEY-BAK SUPER48 X...|Clothing|Bags & A...|
|Keychain - Soccer...|Clothing|Bags & A...|
|Women's Breeze Wa...|Clothing|Shoes|Wo...|
|Crew Diabetic Soc...|Clothing|Women|Wo...|
|Ames Walker AW St...|Clothing|Women|Wo...|
|Gemstone Globe Br...|Clothing|Bags & A...|
|Smart Blonde KC-8...|Clothing|B

In [97]:
distinct_products_belongs_women_category.count()

114

In [98]:
distinct_products_belongs_women_category.toPandas().to_csv('OutputDatas/4. products_belongs_women_category.csv',index= False)

# 5. List the products which are not available.

In [57]:
product_not_available = df.filter(df.Available == 'FALSE').select('Available','Product Name')

In [58]:
product_not_available.show(5)

+---------+--------------------+
|Available|        Product Name|
+---------+--------------------+
|    FALSE|Upper Crust Carib...|
|    FALSE|Fish Tank Ornamen...|
|    FALSE|3-Pack Black Peel...|
|    FALSE|Ames Walker AW St...|
|    FALSE|Turkey Hill Diet ...|
+---------+--------------------+
only showing top 5 rows



In [60]:
product_not_available.count()

8822

In [62]:
product_not_available.distinct().count()

1596

In [63]:
product_not_available = product_not_available.distinct()
product_not_available.show()

+---------+--------------------+
|Available|        Product Name|
+---------+--------------------+
|    FALSE|Produce Houweling...|
|    FALSE|matave organic 20...|
|    FALSE|Heepo Tiger Skin ...|
|    FALSE|Unik Occasions Sp...|
|    FALSE|Country Brook Des...|
|    FALSE|Rapesco 923/12mm ...|
|    FALSE|Buckle-Down PC-W3...|
|    FALSE|Travel Cosmetic M...|
|    FALSE|ConvaTec ActiveLi...|
|    FALSE|DuraGlobe Monthly...|
|    FALSE|British Flag Hear...|
|    FALSE|Solid Dog Polo by...|
|    FALSE|Moleskine Message...|
|    FALSE|comfortable Flex-...|
|    FALSE|Birthday Wishes B...|
|    FALSE|HILASON 1200D RIP...|
|    FALSE|AARCO Enclosed Wa...|
|    FALSE|Gerber Nuk BPA Fr...|
|    FALSE|Splash Home GEL-B...|
|    FALSE|Electric Height-A...|
+---------+--------------------+
only showing top 20 rows



In [64]:
product_not_available.toPandas().to_csv('OutputDatas/5. product_not_available.csv',index= False)

# 6. Count the number of products which are available.

In [67]:
product_available = df.filter(df.Available == 'TRUE').select('Product Name')
product_available.show()

+--------------------+
|        Product Name|
+--------------------+
|In Style Eyes Cat...|
|In Style Eyes Cat...|
|In Style Eyes Cat...|
|In Style Eyes Cat...|
|Replacement for L...|
|American Foxhound...|
|Ironwood Modular ...|
|Classique 766 Pos...|
|Ames Walker AW St...|
|Ames Walker AW St...|
|Ames Walker AW St...|
|Ames Walker AW St...|
|Ames Walker AW St...|
|Ames Walker AW St...|
|Ames Walker AW St...|
|Ames Walker AW St...|
|Ames Walker AW St...|
|Ames Walker AW St...|
|Ames Walker AW St...|
|Ames Walker AW St...|
+--------------------+
only showing top 20 rows



In [70]:
## product available 
product_available.count()

21177

In [71]:
## distinct product available
product_available.distinct().count()

2259

# 7. List the products that are made up of Nylon.

In [76]:
nylon_producs = df.where(col('Description').contains('Nylon') | col('Description').contains('nylon'))
nylon_producs.show(5)

+---------+-----------+--------------------+--------------------+--------------------+------------+-----------+----------+---------------+-----------+--------------------+--------------------+----------+--------------------+---------------+
|Available|      Brand|            Category|     Crawl Timestamp|         Description|        Gtin|Item Number|List Price|   Package Size|Postal Code|        Product Name|         Product Url|Sale Price|             Uniq Id|_corrupt_record|
+---------+-----------+--------------------+--------------------+--------------------+------------+-----------+----------+---------------+-----------+--------------------+--------------------+----------+--------------------+---------------+
|     TRUE|Ames Walker|Clothing|Women|Wo...|2019-03-22 17:46:...|There's a reason ...|649906451211|           |     46.99|size-x-largereg|           |Ames Walker AW St...|https://www.walma...|     14.29|c8a6826ae0a1045cf...|           null|
|     TRUE|Ames Walker|Clothing|Wome

In [83]:
nylon_producs = nylon_producs.select('Product Name', 'Description')
nylon_producs.show(5)

+--------------------+--------------------+
|        Product Name|         Description|
+--------------------+--------------------+
|Ames Walker AW St...|There's a reason ...|
|Ames Walker AW St...|There's a reason ...|
|Ames Walker AW St...|There's a reason ...|
|Ames Walker AW St...|There's a reason ...|
|Ames Walker AW St...|There's a reason ...|
+--------------------+--------------------+
only showing top 5 rows



In [84]:
nylon_producs.count()

551

In [85]:
nylon_producs.distinct().count()

77

In [87]:
distiinct_nylon_products = nylon_producs.distinct()
distiinct_nylon_products.show()

+--------------------+--------------------+
|        Product Name|         Description|
+--------------------+--------------------+
|Country Brook Des...|Country Brook Des...|
|Extra large Profe...|Extra large Profe...|
|Plain Nylon Dog C...|Plain Nylon Dog C...|
|Coastal Pet Nylon...|This lead consist...|
|L'Oreal Paris Inf...|This is the Long-...|
|Country Brook Des...|Country Brook Des...|
|Country Brook Des...|Made by hand in t...|
|Ames Walker AW St...|There's a reason ...|
|Pre-Vent II Serie...|Part #36810500 - ...|
|Unique Bargains A...|Description: It i...|
|Coastal Pet Produ...|Coastal Pet Produ...|
|Anti-Embolism Sto...||Blue Jay Brand *...|
|Unique Bargains P...|Package Content: ...|
|Country Brook Pet...|Half check collar...|
|Doco DCA201-18XL ...|Puffy air mesh st...|
|DJO ProCare Elbow...|DJO ProCare Elbow...|
|2PET Puppy ID Col...| EASY TO WEAR &am...|
|Doco DCROPE2072-0...|Reflective rope l...|
|BELLINGHAM NITRIL...|BELLINGHAM NITRIL...|
|Ames Walker Women...|This new A

In [88]:
distiinct_nylon_products.toPandas().to_csv('OutputDatas/7. nylon_products.csv',index= False)